# Section1: Experiment Search Console


In [1]:
import os,sys,inspect
path=os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
sys.path.append(os.path.abspath(path+"/../../"))
sys.path.append(os.path.abspath(path+"/../../jef/"))
path=os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
sys.path.append(os.path.abspath(path+"/../"))
path=os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
sys.path.append(os.path.abspath(path+"/../components/"))
path=os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
sys.path.append(os.path.abspath(path+"/../dashboards/"))

import SystemConsole
import  BokehComponents
from BokehComponents import *

from jef.experiment_manager import experiment_manager
from conf.conf import conf

Loading BokehJS ...

In [2]:
class ProcessQuery(BufferedQueryInterface):
    def load_data_buffer(self):
        if hasattr(self, 'data'):
            pass
        else:
            self.data = {
                'process_id':[str(i) for i in range(100)],
                'file':['process_name' for i in range(100)],
                'memory':[ i%10+10 for i in range(100)],
                'cpu':[ i%5+10 for i in range(100)],
                }
            self.actions = {'kill':self.action_kill}
    
    def action_kill(self,ids):
        for k in self.data.keys(): 
            for selected_index in reversed(ids):
                self.data[k].pop(selected_index)

class ExperimentManagerInterface:
    
    _em = None
    
    @staticmethod
    def getDataGrid(type_id='experiment'):
        if ExperimentManagerInterface._em is None:
            ExperimentManagerInterface._em=experiment_manager( mongo_server = conf.get( 'jef_mongo_host', '54.214.220.236' ),
                                    port = conf.get( 'jef_mongo_port', 27017 ),
                                    mongo_db = conf.get( 'jef_mongo_db', 'fleetRover' ),
                                    username = conf.get( 'jef_mongo_username', 'pax_user' ),
                                    password = conf.get( 'jef_mongo_password', 'paxuser43' ),
                                    authSource = conf.get( 'jef_mongo_authSource', 'fleetRover' ) )

                            
        df=ExperimentManagerInterface._em.listexpts(status=['running'],query={'settings.experiment_type':type_id})
        L = len(df.index)
        if L==0:
            return {}
        data = {
                'id':[ df.loc[i,'experiment_id'] for i in range(L)],
                'name':[ df.loc[i, 'name'] for i in range(L)],
                'status':['running ' for i in range(L)],
                'duration':[0 for i in range(L)],
                }                   
        return data
        
class ExperimentQuery(BufferedQueryInterface):
 
    def load_data_buffer(self):
 
        self.data = ExperimentManagerInterface.getDataGrid(type_id="experiment")    
        self.actions = {'kill':self.action_kill}
    
    def action_kill(self,ids):
        for k in self.data.keys(): 
            for selected_index in reversed(ids):
                self.data[k].pop(selected_index)

                
class AlgorithmQuery(BufferedQueryInterface):
    def load_data_buffer(self):
        
        self.data = ExperimentManagerInterface.getDataGrid(type_id="algorithm")    
        self.actions = {'kill':self.action_kill} 
    
    def action_kill(self,ids):
        for k in self.data.keys(): 
            for selected_index in reversed(ids):
                self.data[k].pop(selected_index)                
#------- usage    
        
#pi = ProcessQuery()
#df = pd.DataFrame(pi.QueryData())
#df = pd.DataFrame(pi.QueryData(query={'key':'process_id','value':'6'}))
#display(df)
#pi.DoAction(action_id = 'kill', query={'key':'file','value':'jobStatrer.pyc '} )
#pi.DoAction(action_id = 'kill', query={'key':'process_id','value':'6'} )
#df = pd.DataFrame(pi.QueryData())
#display(df)


In [3]:
# Define our buttions
class KillButton(BokehButton):
    def handle_click(self,event):
        for t in self._settings['tables_to_kill']:
            t.removeSelected()


class ShowButton(BokehButton):
    def handle_click(self,event):
        rows = []
        for targetTable in self._settings['analytics_targets']:
            selected = targetTable.getSelected()
            if len(selected) > 0:
                selected = selected[0]
                row = {}
                for col in targetTable.data.keys(): 
                    row[col]=targetTable.data[col][selected]
                rows.append(row)
        ## Load the experiment manager, and get analytics
        
        ad = self._settings['analytics_div']
        ad.text = str(rows)

In [4]:
from datetime import date,datetime,timedelta
from random import randint
from bokeh.io import output_file, show
from bokeh.layouts import widgetbox, Spacer
from bokeh.models import ColumnDataSource,CustomJS,Div
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn, Tabs, Panel
from bokeh.models.widgets import Button, RadioButtonGroup, Select, Slider
from bokeh.layouts import row
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.layouts import layout
from bokeh.events import ButtonClick
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application
import pandas as pd
import pandas_datareader as pdr
from bokeh.plotting import figure,show,output_file

from BokehComponents import *
from SystemConsole import *

output_notebook()
output_file("data_table.html")
class FinancialConsole:
    def __init__(self):
        self.div = Div(width=300)
        self.pt  = QueryTableComponent({'buffered_query_interface':ProcessQuery(),'id_field':'process_id'})        
        self.et  = QueryTableComponent({'buffered_query_interface':ExperimentQuery(),'id_field':'eid'})        
        self.at  = QueryTableComponent({'buffered_query_interface':AlgorithmQuery(),'id_field':'aid'})
        
        self.analytics_div = Div(text="""Analytics HTML""", width=200, height=100)
        self.console = Div(text="""System Out console:""", width=200, height=100)
        
        self.b_kill = KillButton({'tables_to_kill':[self.pt,self.et,self.at],'label':'kill'})
        self.b_show = ShowButton({'label':'show','analytics_targets':[self.et,self.at],'analytics_div':self.analytics_div})
        self.b_restart = ShowButton({'label':'restart'})
        #self.time_graphic = TimeseriesGraphic()
        #self.periodicCallbacks = [self.time_graphic.callback_refresh_analytics]
        self.periodicCallbacks = []
        
        self.s_adviceSelect = BokehSelect({'title':"Advice On:", 'value':"Auto", 'options':["Yes", "No", "Auto"]})
        self.s_tradeSelect = BokehSelect({'title':"Trade:", 'value':"Auto", 'options':["Yes", "No", "Auto"]})
        self.s_autoRestart = BokehSelect({'title':"Autostart:", 'value':"No", 'options':["Yes", "No"]})
        
    def getPeriodicCallbacks(self):
        return self.periodicCallbacks
        
    def getControlPanel(self):
        
        bk = self.b_kill.getBokehComponent()
        br = self.b_restart.getBokehComponent()
        bs = self.b_show.getBokehComponent()
        adviceSelect = self.s_adviceSelect.getBokehComponent()
        tradeSelect  = self.s_tradeSelect.getBokehComponent()
        autoRestart  = self.s_autoRestart.getBokehComponent()
        
        c_tab1 = Panel(child=layout([bk,bs,br], sizing_mode='fixed'),title="Commands")
        c_tab2 = Panel(child=layout([adviceSelect,tradeSelect,autoRestart], sizing_mode='fixed'),title="Signals")
        c_tabs = Tabs(tabs=[ c_tab1,c_tab2], width=300)
        
        return widgetbox(c_tabs)


    def getConsole(self):

        #(Compute Nodes, Running Experiments, Queue and History, Orders & Advice )
        #financePlot = self.time_graphic.getBokehComponent()

        j_tab1 = Panel(child=layout([self.analytics_div], sizing_mode='fixed'),title="Analytics")
        j_tab2 = Panel(child=layout([Div(height=500,width=500)], sizing_mode='fixed'),title="System Out")
        j_tab3 = Panel(child=layout([Div(height=500,width=500)], sizing_mode='fixed'),title="Log Data")
        j_tab4 = Panel(child=layout([Div(height=500,width=500)], sizing_mode='fixed'),title="Job Code")
        j_tabs = Tabs(tabs=[ j_tab1, j_tab2, j_tab3, j_tab4 ], width=700)

        
        l1 = layout([
          [
            [self.pt.getBokehComponent()]         
          ],
           [self.getControlPanel()],
            [self.console]            
        ],sizing_mode='fixed')         
        
        l2 = layout([
          [
            [self.et.getBokehComponent()],
                Spacer(width=20),
           [self.at.getBokehComponent()]          
          ],
           [self.getControlPanel(),j_tabs],
        ],sizing_mode='fixed') 
        

        tab1 = Panel(child=l1,title="System Processes Test")
        tab2 = Panel(child=l2,title="Running Experiments")
        tab3 = Panel(child=Div(height=300,width=600),title="Orders & Integration")
        tabs = Tabs(tabs=[ tab1, tab2, tab3 ])
        return tabs
    
SystemConsole.FinancialConsole = FinancialConsole

Loading BokehJS ...

In [5]:
from conf.conf import conf
SystemConsole.showConsole(conf.get("this_notebook_server","54.184.199.101:9999"))

[1]
[0]
[0]
[0]


In [ ]:
### Job Management
# [ ] select: Queue, Running, Halted, Finihsed
# [ ] search: MongoDB settings query 
# [ ] Parent / Child Filtering
# [ ] Parent = All Jobs present
# [ ] Parent = No Parent
# [X] running experiments in tab 2
# [X] get kills working
# [X] get show button working
# [ ] get sub experiments listing based on experiment
# [ ] autostart property on Experiment
# [ ] See job settings

### Orders & Advice System
# [ ] See linked in traction
# [ ] For each Experiment (running / historical):
# [ ] Sent Brackets + P/L
# [ ] Pending Brackets + P/L
# [ ] Raw P/L
# [ ] Advice (On / Off)
# [ ] Trading (On / Off)



In [ ]:
ee.kill()

In [ ]:
from bokeh.plotting import figure
from bokeh.io import show, output_notebook

# Create a blank figure with labels
p = figure(plot_width = 600, plot_height = 600, 
           title = 'Example Glyphs',
           x_axis_label = 'X', y_axis_label = 'Y')

# Example data
squares_x = [1, 3, 4, 5, 8]
squares_y = [8, 7, 3, 1, 10]
circles_x = [9, 12, 4, 3, 15]
circles_y = [8, 4, 11, 6, 10]

# Add squares glyph
p.square(squares_x, squares_y, size = 12, color = 'navy', alpha = 0.6)
# Add circle glyph
p.circle(circles_x, circles_y, size = 12, color = 'red')

# Set to output the plot in the notebook
output_notebook()
# Show the plot
show(p)

In [ ]:
from pymongo import MongoClient
mf = mongoInfo
display(mf)
client=MongoClient(host=mf['mongo_server'],port=mf['port'],username=mf['username'],password=mf['password'],authSource=mf['authSource'])
display(client)

In [ ]:
display(client.list_database_names())

In [ ]:
str_db = client.list_database_names()[0]
display(str_db)

In [ ]:
display(client[str_db].collection_names())


In [ ]:
import pprint
cur = client['fleetRover']['yahoo'].find().limit(500)
data = []
for f in cur:
    data.append(f)
df = pd.DataFrame(data)
display(df)

In [ ]:
d = df[['High','Low']]
display(d)

In [ ]:
# Read the data from a csv into a dataframe
# Summary stats for the column of interest
d['High'].describe()


In [ ]:
#Develop a Histogram
import numpy as np
arr_hist, edges = np.histogram(d['High'], 
                               bins = int(180/5), 
                               range = [-60, 120])
# Put the information in a dataframe
delays = pd.DataFrame({'arr_delay': arr_hist, 
                       'left': edges[:-1], 
                       'right': edges[1:]})

# Create the blank plot
p = figure(plot_height = 600, plot_width = 600, 
           title = 'Histogram',
          x_axis_label = 'Amount]', 
           y_axis_label = 'Number')

# Add a quad glyph
p.quad(bottom=0, top=d['Low'], 
       left=delays['left'], right=delays['right'], 
       fill_color='red', line_color='black')

# Show the plot
show(p)